# Working with images
In the examples below we demonstrate how to use images with EDSL surveys. 
This involves: 
* Posting and retrieving images at Coop using the `FileStore` module 
* Creating `Scenario` objects for images
* Constructing questions about images
* Selecting vision models to answer the questions
* Running a survey with image scenarios and models

Before running the code below please ensure that you have [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and either activated remote inference or stored your own [API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for language models. Our [docs page](https://docs.expectedparrot.com/) includes a [starter tutorial](https://docs.expectedparrot.com/en/latest/starter_tutorial.html) and other demo notebooks for various use cases. Feel free to email us questions (info@expectedparrot.com) or [post them at our Discord channel](https://discord.com/invite/mxAYkjfy9m).

## Posting images to Coop
We starting by using the `FileStore` module to post some images of parrots to Coop that we have stored locally (source: [Wikipedia: Parrot](https://en.wikipedia.org/wiki/Parrot)):

In [1]:
from edsl import FileStore

image_files = [
    {"filename":"scarlet_macaw_riding_bicycle.jpg", "description":"Scarlet macaw riding a bicycle"},
    {"filename":"sun_conure_parrot_puzzle.jpg", "description":"Sun conure doing a puzzle"},
    {"filename":"rose_ringed_parrot.jpg", "description":"Rose ringed parrot"},
    {"filename":"macaw_parrot_sitting_on_a_tree_branch.jpg", "description":"Macaw sitting on a tree branch"}
]

filestore_images = []

for image_file in image_files:
    fs = FileStore(image_file["filename"])
    info = fs.push(description = image_file["description"], visibility = "public")
    filestore_images.append(info)

filestore_images

[{'description': 'Scarlet macaw riding a bicycle',
  'object_type': 'scenario',
  'url': 'https://www.expectedparrot.com/content/6579f93f-9d50-4d6c-8134-41f6e536d027',
  'alias_url': 'https://www.expectedparrot.com/content/RobinHorton/arrogant-salmon-error',
  'uuid': '6579f93f-9d50-4d6c-8134-41f6e536d027',
  'version': '0.1.62.dev1',
  'visibility': 'public'},
 {'description': 'Sun conure doing a puzzle',
  'object_type': 'scenario',
  'url': 'https://www.expectedparrot.com/content/b30a2db6-0673-410b-ac62-4e0cbb4ab554',
  'alias_url': 'https://www.expectedparrot.com/content/RobinHorton/excellent-excited-histogram',
  'uuid': 'b30a2db6-0673-410b-ac62-4e0cbb4ab554',
  'version': '0.1.62.dev1',
  'visibility': 'public'},
 {'description': 'Rose ringed parrot',
  'object_type': 'scenario',
  'url': 'https://www.expectedparrot.com/content/2aca363e-41b7-4a58-8caf-2dcc46402753',
  'alias_url': 'https://www.expectedparrot.com/content/RobinHorton/green-progressive-partition',
  'uuid': '2aca363

The images are now all posted publicly at Coop.

We can check any image at Coop by retrieving and saving it locally. 
Here we retrieve and resave the files that we posted in the step above:

In [2]:
from edsl import FileStore

for i in range(len(filestore_images)):
    FileStore.pull(filestore_images[i]["uuid"]).write(image_files[i]["filename"])

File written to scarlet_macaw_riding_bicycle.jpg
File written to sun_conure_parrot_puzzle.jpg
File written to rose_ringed_parrot.jpg
File written to macaw_parrot_sitting_on_a_tree_branch.jpg


## Creating scenarios for images
Next we create `Scenario` object for each image in order to use it with survey questions.
This is done by passing the `FileStore` object as the scenario values:

In [3]:
from edsl import Scenario, ScenarioList

scenarios = ScenarioList(
    Scenario({
        "image":FileStore.pull(filestore_image["uuid"]),
        "description":filestore_image["description"]
    }) for filestore_image in filestore_images
)
scenarios

,image,description
0,FileStore: self.path,Scarlet macaw riding a bicycle
1,FileStore: self.path,Sun conure doing a puzzle
2,FileStore: self.path,Rose ringed parrot
3,FileStore: self.path,Macaw sitting on a tree branch


Note that we could also do this by creating scenarios for the local files directly (i.e., if we have not used the `FileStore`):

In [4]:
from edsl import Scenario, ScenarioList

sl = ScenarioList(
    Scenario(i) for i in image_files
)
sl = scenarios.rename({"filename":"image"})
sl

,image,description
0,FileStore: self.path,Scarlet macaw riding a bicycle
1,FileStore: self.path,Sun conure doing a puzzle
2,FileStore: self.path,Rose ringed parrot
3,FileStore: self.path,Macaw sitting on a tree branch


We can confirm these scenarios are identical:

In [5]:
sl == scenarios

True

Next we create a survey of questions using the images. 
Note that we create a `{{ placeholder }}` in the question text for the image, but not the description.
This is because we will attach the description as metadata only--it will be included as a column of the results for reference purposes.
This can be useful when you want to add information about your survey components to avoid dataset match-ups after running the survey (learn more about [using scenarios to add metadata](https://docs.expectedparrot.com/en/latest/scenarios.html#metadata)).

In [6]:
from edsl import QuestionMultipleChoice, QuestionFreeText, Survey

q1 = QuestionMultipleChoice(
    question_name = "identify",
    question_text = "What do you see in this image? {{ scenario.image }}",
    question_options = ["bird", "fox", "horse", "goose"]
)

q2 = QuestionFreeText(
    question_name = "color",
    question_text = "What color is the animal in this image? {{ scenario.image }}"
)

survey = Survey([q1, q2])

# Selecting models
We need to use vision models with image questions (learn more about [choosing language models](https://docs.expectedparrot.com/en/latest/language_models.html) for surveys):

In [7]:
from edsl import Model

model = Model("gemini-1.5-flash", service_name = "google")

# Running a survey 
We run the survey by adding the scenarios and models and then calling the `run()` method on it.
Here we have activated remote inference so the results are automatically stored at Coop:

In [8]:
results = survey.by(scenarios).by(model).run() 

In [9]:
results.select("model", "scenario_index", "description", "identify", "color")

,model.model,scenario.scenario_index,scenario.description,answer.identify,answer.color
0,gemini-1.5-flash,0,Scarlet macaw riding a bicycle,bird,"The animal in the image is a scarlet macaw. Its colors are predominantly red, with yellow and blue feathers as well."
1,gemini-1.5-flash,1,Sun conure doing a puzzle,bird,"That's a Sun Conure. Its color is primarily **orange**, with some yellow and red highlights."
2,gemini-1.5-flash,2,Rose ringed parrot,bird,"The animal in the image is predominantly green, with a vibrant turquoise tail. There is also a small amount of red on its beak."
3,gemini-1.5-flash,3,Macaw sitting on a tree branch,bird,"The animal in the image is primarily blue, with yellow on its chest and shoulders, and black on its beak. There is also a touch of green on its head."


## Posting to Coop
The results of the survey have been automatically posted to Coop.
Here we also post this notebook:

In [ ]:
from edsl import Notebook

nb = Notebook(path = "image_examples.ipynb")

nb.push(
    description = "Image examples", 
    alias = "image-examples",
    visibility = "public"
)